In [ ]:
import os, sys
import numpy as np
import ants
import argparse
import shutil
import logging
import subprocess
import glob
from tqdm import tqdm
import matplotlib.pyplot as plt 
import pandas as pd

In [ ]:
from glob2 import glob

path_gbm = '/anvar/public_datasets/preproc_study/gbm/1_reg'
path_bgpd = '/anvar/public_datasets/preproc_study/bgpd/orig'
path_schw = '/anvar/public_datasets/preproc_study/schw/orig'

list_gbm = glob(path_gbm + '/**/CT1_SEG.nii.gz')
list_bgpd = glob(path_bgpd + '/**/mask_GTV_FLAIR.nii.gz')
list_schw = glob(path_schw + '/**/T1_SEG.nii.gz')

In [26]:
# checking for right labels
import ants

for img_path in tqdm(list_gbm):
    img_orig = ants.image_read(img_path)
    channels = np.unique(img_orig.numpy())[1:]
    if channels[0] != 1:
        print(img_path)
        print('weird channels', channels)
        result_arr = img_orig.numpy()
        result_arr[result_arr == channels[0]] = int(1)
        result_arr[result_arr == channels[1]] = int(2)
        result_arr[result_arr == channels[2]] = int(3)
        img_new = img_orig.new_image_like(result_arr)
        print(np.unique(img_new.numpy())[1:])
#         os.remove(img_path)
#         ants.image_write(img_new, img_path, ri=False);

 19%|█▊        | 19/102 [00:05<00:14,  5.69it/s]

/anvar/public_datasets/preproc_study/gbm/1_reg/TCGA-76-6661/CT1_SEG.nii.gz
weird channels [1.560928 4.682784 6.243712]
[1. 2. 3.]


 46%|████▌     | 47/102 [00:15<00:24,  2.25it/s]

/anvar/public_datasets/preproc_study/gbm/1_reg/TCGA-14-1794/CT1_SEG.nii.gz
weird channels [2.5308914 5.061783  7.5926743]


 47%|████▋     | 48/102 [00:15<00:25,  2.09it/s]

...

 78%|███████▊  | 80/102 [00:28<00:04,  4.78it/s]

/anvar/public_datasets/preproc_study/gbm/1_reg/TCGA-76-4934/CT1_SEG.nii.gz
weird channels [1.524298 3.048596 4.572894]
[1. 2. 3.]


 95%|█████████▌| 97/102 [00:38<00:02,  1.76it/s]

/anvar/public_datasets/preproc_study/gbm/1_reg/TCGA-76-6656/CT1_SEG.nii.gz
weird channels [1.9323565 3.864713  5.7970695]
[1. 2. 3.]


100%|██████████| 102/102 [00:39<00:00,  2.56it/s]


### Quality check:

In [39]:
base_dir = '/anvar/public_datasets/preproc_study/bgpd/4a_resamp/'
mask = 'mask_GTV_FLAIR.nii.gz'

for patient in tqdm(os.listdir(base_dir)):
    if not patient == 'logging.txt':
        for modality in ['T1.nii.gz','T2.nii.gz','FLAIR.nii.gz','CT1.nii.gz', mask]:
            img = ants.image_read('/anvar/public_datasets/preproc_study/bgpd/4a_resamp/{}/{}'.format(patient, modality))
            
            if modality == 'T1.nii.gz':
                t1_shape = np.shape(img.numpy())
                
            if modality == 'T2.nii.gz':
                t2_shape = np.shape(img.numpy())
                
            if modality == 'FLAIR.nii.gz':
                flair_shape = np.shape(img.numpy())
                
            # test for binary mask
            if modality == mask:
                # for 1 channel labels
                if len(np.unique(img.numpy())) > 2:
                    print(patient, np.unique(img.numpy()))
                mask_shape = np.shape(img.numpy())
            # test for nans    
            if not img.sum() > 0:
                print(patient, modality, 'Amount of nans:', np.shape(np.argwhere(np.isnan(img.numpy()))))
        
        # test for shapaes
        if (t1_shape != t2_shape)| (t1_shape != flair_shape) | (t1_shape != mask_shape):
            print(patient, ' t1', t1_shape, ' t2', t2_shape, 'flair', flair_shape, ' mask', mask_shape)

100%|██████████| 181/181 [05:52<00:00,  1.95s/it]


In [ ]:
base_dir = '/anvar/public_datasets/preproc_study/gbm/6_hist/6_hist_fold_1/'
predicate_dir = '/anvar/public_datasets/preproc_study/gbm/4a_resamp/'
mask = 'CT1_SEG.nii.gz'
label = [0,1,2,3]

def check_path(base_dir, predicate_dir, mask, label):
    for patient in tqdm(os.listdir(base_dir)):
        if not patient == 'logging.txt':
            for modality in ['T1.nii.gz','T2.nii.gz','FLAIR.nii.gz','CT1.nii.gz', mask]:
                img = ants.image_read(base_dir + '/{}/{}'.format(patient, modality))

                if modality == 'T1.nii.gz':
                    t1_shape = np.shape(img.numpy())

                    #checking for T1 only, if the data remains unchanged
                    img_pred = ants.image_read(predicate_dir + '/{}/{}'.format(patient, modality))
                    if np.allclose(img.numpy(),img_pred.numpy()):
                        print('The sourse data and preprocessed are equal')

                if modality == 'T2.nii.gz':
                    t2_shape = np.shape(img.numpy())

                if modality == 'FLAIR.nii.gz':
                    flair_shape = np.shape(img.numpy())

                # test for binary mask
                if modality == mask:
                    # for 1 channel labels
                    if len(np.unique(img.numpy())) > 2:
                        if (np.unique(img.numpy()) != label).all():
                            print(patient, np.unique(img.numpy()))
                            break
                    mask_shape = np.shape(img.numpy())
                # test for nans    
                if not img.sum() > 0:
                    print(patient, modality, 'Amount of nans:', np.shape(np.argwhere(np.isnan(img.numpy()))))

            # test for shapaes
            if (t1_shape != t2_shape)| (t1_shape != flair_shape) | (t1_shape != mask_shape):
                print(patient, ' t1', t1_shape, ' t2', t2_shape, 'flair', flair_shape, ' mask', mask_shape)

In [ ]:
1

In [ ]:
base_dir = '/anvar/public_datasets/preproc_study/gbm/5_ss_shared'
predicate_dir = '/anvar/public_datasets/preproc_study/gbm/4a_resamp/'
mask = 'CT1_SEG.nii.gz'
label = [0,1,2,3]

check_path(base_dir, predicate_dir, mask, label)

In [53]:
os.listdir('/anvar/public_datasets/preproc_study/gbm/6_hist/6_hist_fold_0/TCGA-02-0086')

['T2.nii.gz', 'FLAIR.nii.gz', 'T1.nii.gz', 'CT1.nii.gz']

In [ ]:
nohup python 5_ss.py --path /anvar/public_datasets/preproc_study/gbm/4a_resamp/ --output /mnt/public_data/preproc_study/gbm/5_ss_shared/ --device 0 &